In [1]:
import pandas as pd
import pm4py
#Import right event log
df = pd.read_csv("benchmark.csv" ,delimiter=';' )
#import wrong event log
df_wrong = pd.read_csv("incorrect_logGENA.csv",delimiter=';')
og_columns = list(df_wrong.columns)[1:]
print(og_columns)
df_wrong.head(5)[og_columns]


['concept:name', 'time:timestamp', 'case:concept:name', 'pName']


,concept:name,time:timestamp,case:concept:name,pName
0,receive application,2021-11-03 00:54:23 +01:00,Trace 1,Hans
1,declined GDPR,2021-11-03 02:27:19 +01:00,Trace 1,Hans
2,notify recruiter,2021-11-03 04:02:51 +01:00,Trace 1,Hans
3,review application,2021-11-03 05:25:21 +01:00,Trace 1,Hans
4,schedule interview and send invitation,2021-11-03 06:33:58 +01:00,Trace 1,Hans


In [2]:
## csv
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.statistics.traces.generic.log import case_statistics
from pm4py.objects.conversion.log import converter as log_converter

# get all variants from correct data
log_csv = dataframe_utils.convert_timestamp_columns_in_df(df)
log_csv = log_csv.sort_values('time:timestamp')
event_log = log_converter.apply(log_csv)
variants_count = case_statistics.get_variant_statistics(event_log)
variants = []
for x in variants_count:
    variants += [x['variant']]
    
# get all variants from wrong data
log_csv2 = dataframe_utils.convert_timestamp_columns_in_df(df_wrong)
log_csv2 = log_csv2.sort_values('time:timestamp')
event2_log = log_converter.apply(log_csv2)
wrong_variants_count = case_statistics.get_variant_statistics(event2_log)
wrong_variants = []
for x in wrong_variants_count:
    wrong_variants += [x['variant']]

In [3]:
#which true variants are avaiable //test
variants
len(variants_count)

8

In [4]:
#difference of variants //test
variants_diff = list(set(wrong_variants) - set(variants))
len(variants_diff)

23

In [5]:
#looking at difference of variable //test
print(f"true variant count: {len(variants)}\nwrong variant count: {len(wrong_variants)}")

true variant count: 8
wrong variant count: 31


In [6]:
# filter on gdpr (confirm GDPR, decline GDPR, x)
filtered_log = pm4py.filter_directly_follows_relation(event2_log, [('confirm GDPR','decline GDPR')], retain=True)
print(len(filtered_log))
filtered_log = pm4py.filter_end_activities(filtered_log, ['decline GDPR'], retain=False)
print(len(filtered_log))

#mfiltering wrong traces with GDPR case from above
wrong_traces_gdpr_cd = []
for e in filtered_log:
    wrong_traces_gdpr_cd += [e.attributes['concept:name']]
    
wrong_traces_gdpr_cd[:5]

2047
1028


['Trace 3736', 'Trace 3031', 'Trace 3603', 'Trace 3075', 'Trace 2017']

In [7]:
#marking wrong traces with a reason
df_wrong['error_reason_3'] = ''
for x in wrong_traces_gdpr_cd:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'decline GDPR')].index[0]+1, 
                 ['error_reason_3']] = '3' #GDPR Breach


In [8]:
# filter on gdpr (reject application)
filtered_log = pm4py.filter_directly_follows_relation(event2_log, [('decline permission', 'save data')], retain=True)

print(len(filtered_log))

wrong_traces_gdpr_ra = []
for e in filtered_log:
    wrong_traces_gdpr_ra += [e.attributes['concept:name']]
    
wrong_traces_gdpr_ra[:5]
#write down reason
for x in wrong_traces_gdpr_ra:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'save data')].index[0], 
                 ['error_reason_3']] = '3' #GDPR Breach

54


In [9]:
pm4py.get_event_attribute_values(event2_log, 'concept:name')

{'receive application': 10000,
 'confirm GDPR': 8000,
 'confirmed GDPR': 3962,
 'notify recruiter': 5511,
 'review application': 5511,
 'delete data of applicant': 2660,
 'decline GDPR': 2047,
 'schedule interview and send invitation': 2851,
 'await answer\nfor interview': 504,
 'hold intervierw': 1139,
 'send to assessment center': 615,
 'create and send offer': 328,
 'wait for answer to offer': 322,
 'Hire candidate': 105,
 'begin onboarding process': 132,
 'reject application/\nask for permission to save data': 688,
 'await answer of applicant': 133,
 'decline permission': 244,
 'delete data': 256,
 'Applicant rejected/\ndelete data': 749,
 'decline applicant in assesment center': 287,
 'receive permission': 249,
 'save data': 237,
 'declined GDPR': 3991,
 'await answer': 2706,
 'hold interview': 255,
 'applicant rejected/\ndelete data': 228,
 'hire candidate': 27,
 'receive permission/\nsave data': 28,
 'decline premission/\ndelete data': 33}

In [10]:
# filter on negative answer/no process allowed ('review application', 'delete data of applicant', x)
filtered_log = pm4py.filter_directly_follows_relation(event2_log, [('review application', 'delete data of applicant')], retain=True)
print(len(filtered_log))
filtered_log = pm4py.filter_end_activities(filtered_log, ['delete data of applicant'], retain=False)
print(len(filtered_log))

wrong_traces_gdpr_aa = []
for e in filtered_log:
    wrong_traces_gdpr_aa += [e.attributes['concept:name']]
    
wrong_traces_gdpr_aa[:5]

df_wrong['error_reason_4'] = ''
for x in wrong_traces_gdpr_aa:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'delete data of applicant')].index[0], 
                 ['error_reason_4']] = '4' 

print(wrong_traces_gdpr_aa)


2660
483
['Trace 3736', 'Trace 3075', 'Trace 2821', 'Trace 3316', 'Trace 2839', 'Trace 2301', 'Trace 2560', 'Trace 2711', 'Trace 2376', 'Trace 3432', 'Trace 2703', 'Trace 3106', 'Trace 2123', 'Trace 2346', 'Trace 2605', 'Trace 2165', 'Trace 2113', 'Trace 2127', 'Trace 2472', 'Trace 3580', 'Trace 3412', 'Trace 3740', 'Trace 3214', 'Trace 2226', 'Trace 2117', 'Trace 3280', 'Trace 3919', 'Trace 3284', 'Trace 3193', 'Trace 2666', 'Trace 2791', 'Trace 2223', 'Trace 3959', 'Trace 2569', 'Trace 2525', 'Trace 3489', 'Trace 3799', 'Trace 3986', 'Trace 2701', 'Trace 2743', 'Trace 3694', 'Trace 2868', 'Trace 2126', 'Trace 2414', 'Trace 3620', 'Trace 3611', 'Trace 3108', 'Trace 2491', 'Trace 2437', 'Trace 2420', 'Trace 3715', 'Trace 2473', 'Trace 3772', 'Trace 3083', 'Trace 2043', 'Trace 2354', 'Trace 3438', 'Trace 2033', 'Trace 2383', 'Trace 2135', 'Trace 3751', 'Trace 3254', 'Trace 3357', 'Trace 2019', 'Trace 2015', 'Trace 3111', 'Trace 2190', 'Trace 3957', 'Trace 3015', 'Trace 3842', 'Trace 314

In [11]:

# filter on negative answer/no process allowed ('send to assessment center', 'decline application', x)
filtered_log = pm4py.filter_directly_follows_relation(event2_log, [('send to assessment center', 'decline application')], retain=True)
print(len(filtered_log))
filtered_log = pm4py.filter_end_activities(filtered_log, ['decline application'], retain=False)
print(len(filtered_log))

wrong_traces_gdpr_aa = []
for e in filtered_log:
    wrong_traces_gdpr_aa += [e.attributes['concept:name']]
    
wrong_traces_gdpr_aa[:5]

df_wrong['error_reason_6'] = ''
for x in wrong_traces_gdpr_aa:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'delete data of applicant')].index[0], 
                 ['error_reason_6']] = '4' 

print(wrong_traces_gdpr_aa)


0
0
[]


In [12]:
# filter on negative answer/no process allowed ('send to assessment center', 'decline application', x)
filtered_log = pm4py.filter_end_activities(filtered_log, ['wait for an answer'], retain=False)
print(len(filtered_log))

wrong_traces_gdpr_aa = []
for e in filtered_log:
    wrong_traces_gdpr_aa += [e.attributes['concept:name']]
    
wrong_traces_gdpr_aa[:5]

df_wrong['error_reason_7'] = ''
for x in wrong_traces_gdpr_aa:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'delete data of applicant')].index[0], 
                 ['error_reason_7']] = '4' 

print(wrong_traces_gdpr_aa)

0
[]


In [13]:
# filter on negative answer/no process allowed ('await answer', 'Applicant rejected delete data', x)
filtered_log = pm4py.filter_directly_follows_relation(event2_log, [('await answer', 'Applicant rejected/\ndelete data')], retain=True)
print(len(filtered_log))
filtered_log = pm4py.filter_end_activities(filtered_log, ['Applicant rejected/\ndelete data'], retain=False)
print(len(filtered_log))

wrong_traces_gdpr_aa = []
for e in filtered_log:
    wrong_traces_gdpr_aa += [e.attributes['concept:name']]
    
wrong_traces_gdpr_aa[:5]

df_wrong['error_reason_5'] = ''
for x in wrong_traces_gdpr_aa:
    df_wrong.loc[df_wrong.loc[(df_wrong['case:concept:name'] == x) & (df_wrong['concept:name'] == 'Applicant rejected/\ndelete data')].index[0], 
                 ['error_reason_5']] = '4'


499
0


In [14]:
# filter on variants
from pm4py.algo.filtering.log.variants import variants_filter
filtered_log1 = variants_filter.apply(event2_log, variants_diff)
wrong_traces  = []
for e in filtered_log1:
    wrong_traces += [e.attributes['concept:name']]

In [15]:
#marking wrong variants
di_traces = {}
re_traces = {}
#marking wrong variants 
for i in range(len(df_wrong)):
    di_traces["Trace " + str(i)] = ''
    re_traces["Trace " + str(i)] = ''

for e in wrong_traces:
    di_traces[e] = 'x'
    re_traces[e] = '1'   #wrong variant
    
df_wrong['error_case'] = df_wrong['case:concept:name'].replace(di_traces)
df_wrong['error_reason'] = df_wrong['case:concept:name'].replace(re_traces)

print(f"Number of error case 1: {len(wrong_traces)}")
df_wrong.head(5)

Number of error case 1: 7287


,ID,concept:name,time:timestamp,case:concept:name,pName,error_reason_3,error_reason_4,error_reason_6,error_reason_7,error_reason_5,error_case,error_reason
0,0,receive application,2021-11-02 23:54:23+00:00,Trace 1,Hans,,,,,,x,1
1,1,declined GDPR,2021-11-03 01:27:19+00:00,Trace 1,Hans,,,,,,x,1
2,2,notify recruiter,2021-11-03 03:02:51+00:00,Trace 1,Hans,,,,,,x,1
3,3,review application,2021-11-03 04:25:21+00:00,Trace 1,Hans,,,,,,x,1
4,4,schedule interview and send invitation,2021-11-03 05:33:58+00:00,Trace 1,Hans,,,,,,x,1


In [16]:
# filter for wrong activities, names
p = df['concept:name'].unique()
n = df['pName'].unique()

p_wrong = df_wrong['concept:name'].unique()
n_wrong = df_wrong['pName'].unique()

wrong_p = set(p_wrong)-set(p)
wrong_n = set(n_wrong)-set(n)

re = {}
pe = {}

for i in wrong_p:
    re[i] = '5'  #process not existent
for i in p:
    re[i] = ''


for i in wrong_n:
    pe[i] = '2'    #employee not allowed to handle data
for i in n:
    pe[i] = ''
    
    
# create new columns
df_wrong['error_reason_1'] =  df_wrong['concept:name'].replace(re)
df_wrong['error_reason_2'] = df_wrong['pName'].replace(pe)

df_wrong.head(5)


s2 = set()
s5 = set()
        
for ind in df_wrong.index:
    if(df_wrong['pName'][ind] not in n):
        s2.add(df_wrong["case:concept:name"][ind])
        
    if(df_wrong['concept:name'][ind] not in p):
        s5.add(df_wrong["case:concept:name"][ind])
        
print(f"Number of error case 2: {len(s2)}")
print(f"Number of error case 5: {len(s5)}")

Number of error case 2: 456
Number of error case 5: 5263


In [17]:
#show table //test
print (s5)

print(f"Number of error case 5: {wrong_n}")

{'Trace 6769', 'Trace 5892', 'Trace 6176', 'Trace 5078', 'Trace 6485', 'Trace 5009', 'Trace 849', 'Trace 1042', 'Trace 1853', 'Trace 7737', 'Trace 7831', 'Trace 1390', 'Trace 3302', 'Trace 751', 'Trace 7848', 'Trace 530', 'Trace 567', 'Trace 4193', 'Trace 4225', 'Trace 7693', 'Trace 1935', 'Trace 6873', 'Trace 774', 'Trace 439', 'Trace 5308', 'Trace 5460', 'Trace 1723', 'Trace 6981', 'Trace 196', 'Trace 1617', 'Trace 2743', 'Trace 5823', 'Trace 2610', 'Trace 4239', 'Trace 3151', 'Trace 7499', 'Trace 284', 'Trace 5976', 'Trace 6243', 'Trace 2280', 'Trace 7401', 'Trace 2006', 'Trace 1419', 'Trace 7282', 'Trace 5933', 'Trace 1916', 'Trace 1246', 'Trace 1776', 'Trace 4294', 'Trace 2147', 'Trace 7739', 'Trace 7026', 'Trace 1987', 'Trace 7635', 'Trace 5388', 'Trace 5554', 'Trace 7260', 'Trace 4670', 'Trace 6634', 'Trace 1191', 'Trace 4152', 'Trace 806', 'Trace 7772', 'Trace 7925', 'Trace 1778', 'Trace 5619', 'Trace 1140', 'Trace 7300', 'Trace 5709', 'Trace 6474', 'Trace 6724', 'Trace 1378', 

In [18]:
wrong_p

{'applicant rejected/\ndelete data',
 'await answer',
 'decline premission/\ndelete data',
 'declined GDPR',
 'hire candidate',
 'hold interview',
 'receive permission/\nsave data'}

In [19]:
#merge error case and reasons
for ind in df_wrong.index:
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_1'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_1'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_1'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_1'][ind]

    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_2'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_2'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_2'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_2'][ind]
    
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_3'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_3'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_3'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_3'][ind]
        
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_4'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_4'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_4'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_4'][ind]
        
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_5'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_5'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_5'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_5'][ind]
        
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_6'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_6'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_6'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_6'][ind]
        
        
    if (df_wrong['error_reason'][ind] == '' and df_wrong['error_reason_7'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason_7'][ind]
    elif (df_wrong['error_reason'][ind] != '' and df_wrong['error_reason_7'][ind] != ''):
        df_wrong.loc[ind, ['error_reason']] = df_wrong['error_reason'][ind] + ', ' + df_wrong['error_reason_7'][ind]   
        
        
    if(df_wrong['pName'][ind] not in n):
        df_wrong.loc[ind, ['error_case']] = 'x'        

In [20]:
# df_wrong = df_wrong.drop(columns=['error_reason_1', 'error_reason_2', 'error_reason_3', 'x1', ])
df_wrong = df_wrong[og_columns + ['error_case', 'error_reason']]
df_wrong.head(25)

,concept:name,time:timestamp,case:concept:name,pName,error_case,error_reason
0,receive application,2021-11-02 23:54:23+00:00,Trace 1,Hans,x,1
1,declined GDPR,2021-11-03 01:27:19+00:00,Trace 1,Hans,x,"1, 5"
2,notify recruiter,2021-11-03 03:02:51+00:00,Trace 1,Hans,x,1
3,review application,2021-11-03 04:25:21+00:00,Trace 1,Hans,x,1
4,schedule interview and send invitation,2021-11-03 05:33:58+00:00,Trace 1,Hans,x,1
5,await answer,2021-11-03 07:24:34+00:00,Trace 1,Hans,x,"1, 5"
6,Applicant rejected/\ndelete data,2021-11-03 09:11:47+00:00,Trace 1,Hans,x,1
7,receive application,2021-11-03 00:27:40+00:00,Trace 2,Peter,x,1
8,declined GDPR,2021-11-03 01:42:30+00:00,Trace 2,Peter,x,"1, 5"
9,notify recruiter,2021-11-03 03:35:05+00:00,Trace 2,Peter,x,1


In [21]:
df_wrong.to_csv('Wrong_logPLGmarked.csv', sep = ';')

PermissionError: [Errno 13] Permission denied: 'Wrong_logPLGmarked.csv'